In [281]:
#Source Code : #from https://www.kaggle.com/code/andrej0marinchenko/titanic-data-science-solutions-upd/notebook


# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [283]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

In [284]:
train_data = pd.read_csv('/kaggle/input/titanic/train.csv')
x_train = pd.read_csv('/kaggle/input/titanic/train.csv')
x_test = pd.read_csv('/kaggle/input/titanic/test.csv')
y_train = np.array(x_train['Survived'].copy())
id_test = np.array(x_test['PassengerId'].copy())
all_data = [x_train,x_test]

In [285]:
x_train.head()

In [286]:
x_test.head()


In [287]:
#count the number of elements in the dataset
print(x_train.size)

#show number of dimensions of the df
print(x_train.ndim)

#show dataset dimensions 
print(x_train.shape)

#show columns names
print(x_train.columns.values)

#count non null values and show column type
print(x_train.info(verbose=True))

In [288]:
#count the number of elements in the dataset
print(x_test.size)

#show number of dimensions of the df
print(x_test.ndim)

#show dataset dimensions 
print(x_test.shape)

#show columns names
print(x_test.columns.values)

#count non null values and show column type
print(x_test.info(verbose=True))

In [289]:
#show duplicated rows
x_train[x_train.duplicated()]

In [290]:
#show duplicated rows
x_test[x_test.duplicated()]

In [291]:
x_train.describe()

In [292]:
x_train.describe(include=['O'])

In [293]:
x_train.Survived.value_counts().plot.bar()

In [294]:
x_train['Pclass'].value_counts().plot(kind='pie', autopct='%.2f')


In [295]:
x_train['Sex'].value_counts().plot(kind='pie', autopct='%.2f') 


In [296]:
x_train.Embarked.value_counts().plot.bar()


In [297]:
x_train[['Sex', 'Survived']].groupby(['Sex'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [298]:
x_train[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [299]:
x_train[["SibSp", "Survived"]].groupby(['SibSp'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [300]:
x_train[["Parch", "Survived"]].groupby(['Parch'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [301]:
x_train[["Embarked", "Survived"]].groupby(['Embarked'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [302]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.distplot(x_train[x_train['Survived']==0]['Age'])
sns.distplot(x_train[x_train['Survived']==1]['Age'])
plt.legend(labels=["Survived=0","Survived=1"])
plt.show()


In [303]:
grid = sns.FacetGrid(x_train, col='Survived', row='Pclass', size=2.2, aspect=1.6)
grid.map(plt.hist, 'Age', alpha=.5, bins=20)
grid.add_legend();

In [304]:
grid = sns.FacetGrid(x_train, row='Embarked', col='Survived', size=2.2, aspect=1.6)
grid.map(sns.barplot, 'Sex', 'Fare', alpha=.5, ci=None)
grid.add_legend()

In [305]:
x_train = x_train.drop(['Ticket', 'Cabin'], axis=1)
x_test = x_test.drop(['Ticket', 'Cabin'], axis=1)
all_data = [x_train,x_test]

In [306]:
for dataset in all_data:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)

pd.crosstab(x_train['Title'], x_train['Sex'])

In [307]:
for dataset in all_data:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    
x_train[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()

In [308]:
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
for dataset in all_data:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)

x_train.head()

In [309]:
x_train = x_train.drop(['Name', 'PassengerId'], axis=1)
x_test = x_test.drop(['Name'], axis=1)
all_data = [x_train,x_test]

In [310]:
for dataset in all_data:
    dataset['Sex'] = dataset['Sex'].map( {'female': 1, 'male': 0} ).astype(int)

x_train.head()

In [311]:
grid = sns.FacetGrid(x_train, row='Pclass', col='Sex', size=2.2, aspect=1.6)
grid.map(plt.hist, 'Age', alpha=.5, bins=20)
grid.add_legend()

In [312]:
guess_ages = np.zeros((2,3))
guess_ages

for dataset in all_data:
    for i in range(0, 2):
        for j in range(0, 3):
            guess_df = dataset[(dataset['Sex'] == i) & \
                                  (dataset['Pclass'] == j+1)]['Age'].dropna()

    

            age_guess = guess_df.median()

            # Convert random age float to nearest .5 age
            guess_ages[i,j] = int( age_guess/0.5 + 0.5 ) * 0.5
            
    for i in range(0, 2):
        for j in range(0, 3):
            dataset.loc[ (dataset.Age.isnull()) & (dataset.Sex == i) & (dataset.Pclass == j+1),\
                    'Age'] = guess_ages[i,j]

    dataset['Age'] = dataset['Age'].astype(int)

x_train.head()

In [313]:
x_train['AgeBand'] = pd.cut(x_train['Age'], 5)
x_train[['AgeBand', 'Survived']].groupby(['AgeBand'], as_index=False).mean().sort_values(by='AgeBand', ascending=True)

In [314]:
for dataset in all_data:    
    dataset.loc[ dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age']
x_test.head()

In [315]:
x_train  = x_train.drop(['AgeBand'], axis=1)
all_data = [x_train , x_test]
x_train.head()

In [316]:
for dataset in all_data:
    dataset['FamiSize'] = dataset['SibSp'] + dataset['Parch'] + 1

x_train.head(15)

In [317]:
x_train [['FamiSize', 'Survived']].groupby(['FamiSize'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [318]:

x_train['IsAlone'] = 0
x_train.loc[x_train['FamiSize'] == 1, 'IsAlone'] = 1
  
x_test['IsAlone'] = 0
x_test.loc[x_test['FamiSize'] == 1, 'IsAlone'] = 1
x_train [['IsAlone', 'Survived']].groupby(['IsAlone'], as_index=False).mean()

In [319]:
x_train.head()

In [320]:
x_train  = x_train.drop(['Parch', 'SibSp', 'FamiSize'], axis=1)
x_test  = x_test.drop(['Parch', 'SibSp', 'FamiSize'], axis=1)
all_data = [x_train,x_test]

x_train.head()

In [321]:
for dataset in all_data:
    dataset['Age*Class'] = dataset.Age * dataset.Pclass

x_train.loc[:, ['Age*Class', 'Age', 'Pclass']].head(10)

In [322]:
freq_port = x_train.Embarked.dropna().mode()[0]
freq_port

In [323]:
for dataset in all_data:
    dataset['Embarked'] = dataset['Embarked'].fillna(freq_port)
    
x_train[['Embarked', 'Survived']].groupby(['Embarked'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [324]:
for dataset in all_data :
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

x_train.head()

In [325]:
x_test['Fare'].fillna(x_test['Fare'].dropna().median(), inplace=True)
x_test.head()

In [326]:
x_train['FareBand'] = pd.qcut(x_train['Fare'], 4)
x_train[['FareBand', 'Survived']].groupby(['FareBand'], as_index=False).mean().sort_values(by='FareBand', ascending=True)

In [327]:
for dataset in all_data :
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)

x_train = x_train.drop(['FareBand'], axis=1)

    
x_train.head(10)

In [341]:
x_train.head()

In [339]:
x_train = x_train.drop("Survived", axis=1)

In [340]:
x_test.head()

In [336]:
x_test  = x_test.drop("PassengerId", axis=1).copy()

In [342]:
logreg = LogisticRegression()
logreg.fit(x_train , y_train )
Y_pred = logreg.predict(x_test)
acc_log = round(logreg.score(x_train , y_train) * 100, 2)
acc_log

In [343]:
predictss = pd.DataFrame(Y_pred)
predictss.rename( columns={0 :'Survived'}, inplace=True )
x_tests = pd.read_csv('/kaggle/input/titanic/test.csv')

ffff = pd.concat([x_tests['PassengerId'],predictss],axis=1)
final = ffff[['PassengerId','Survived']]

In [344]:
from IPython.display import HTML

final.to_csv('submission.csv', index=False)

def create_download_link(title = "Download CSV file", filename = "data.csv"):  
    html = '<a href={filename}>{title}</a>'
    html = html.format(title=title,filename=filename)
    return HTML(html)

# create a link to download the dataframe which was saved with .to_csv method
create_download_link(filename='submission.csv')